Holy Lovenia / 13515113

# Sentence Selection

### Read and preprocess train data

In [1]:
from nltk import RegexpTokenizer
from pandas.io.json import json_normalize


import json
import pandas as pd
import re

In [3]:
data = []
with open('../dataset/SQuAD/train-v2.0.json') as f:
    json_data = json.load(f)['data']

    for i in range(len(json_data)):
        json_data_i = json_data[i]['paragraphs']
        
        for j in range(1):
            paragraph = json_data_i[j]['context']
                
            data.append(paragraph)

In [7]:
preprocessed_data = []
with open('../dataset/SQuAD/train-v2.0.json') as f:
    json_data = json.load(f)['data']

    for i in range(len(json_data)):
        json_data_i = json_data[i]['paragraphs']
        
        for j in range(1):
            paragraph = json_data_i[j]['context']
            
            # replace all dictionary phonetic with ''
            paragraph = re.sub('\/.*\ˈ.*\/', '', paragraph)
            
            # replace all japanese characters with ''
            paragraph = re.sub('[\u3000-\u303f\u3040-\u309f\u30a0-\u30ff\uff00-\uff9f\u4e00-\u9faf\u3400-\u4dbf]+', '', paragraph)
            
            # replace dots in the center of words
            words = paragraph.split(' ')
            for i in range(len(words)):
                if(words[i].find('.') != len(words[i]) - 1 and words[i].find('.') != -1):
                    words[i] = words[i].replace('.', '')
                if(words[i].find(',') != len(words[i]) - 1 and words[i].find(',') != -1):
                    words[i] = words[i].replace(',', '')

            paragraph = ' '.join(words)
            
            data_i_j = paragraph.split('.')
            
            paragraph = []
            for k in range(len(data_i_j)):
                tokenizer = RegexpTokenizer('[\w\/\&\-\:]+', flags=re.UNICODE)
    
                token_list = tokenizer.tokenize(data_i_j[k])
                token_list = [token for token in token_list if len(token) > 1 or token.lower() == 'a']
            
                if token_list != []:
                    paragraph.append(token_list)
                
            preprocessed_data.append(paragraph)

In [8]:
for i in preprocessed_data:
    sentences = ''
    for j in i:
        sentences = sentences + '\n' + ' '.join(j)
        
    print(sentences)
    print()


Beyoncé Giselle Knowles-Carter bee-YON-say born September 1981 is an American singer songwriter record producer and actress
Born and raised in Houston Texas she performed in various singing and dancing competitions as a child and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny Child
Managed by her father Mathew Knowles the group became one of the world best-selling girl groups of all time
Their hiatus saw the release of Beyoncé debut album Dangerously in Love 2003 which established her as a solo artist worldwide earned five Grammy Awards and featured the Billboard Hot 100 number-one singles Crazy in Love and Baby Boy


Frédéric François Chopin French pronunciation: fʁedeʁik fʁɑ swa ʃɔpɛ 22 February or March 1810 17 October 1849 born Fryderyk Franciszek Chopin was a Polish and French by citizenship and birth of father composer and a virtuoso pianist of the Romantic era who wrote primarily for the solo piano
He gained and has maintained renown worldwide as one of

### TextRank

In [9]:
from nltk.cluster.util import cosine_distance
from nltk.corpus import stopwords
from operator import itemgetter

import numpy as np

In [10]:
def sentence_similarity(sentence1, sentence2, stopwords=None):
    """
    sentence1 and sentence2 = ['word1', 'word2', ...]
    """
    
    if stopwords is None:
        stopwords = []
        
    sentence1 = [word.lower() for word in sentence1]
    sentence2 = [word.lower() for word in sentence2]
    all_words = list(set(sentence1 + sentence2))
    
    vector1 = [0] * len(all_words)
    vector2 = vector1.copy()
    
    # 1st sentence's vector building
    for word in sentence1:
        if word in stopwords:
            continue # do nothing
        
        # add 1 to vector's index corresponding to word
        vector1[all_words.index(word)] += 1
    
    # 2nd sentence's vector building
    for word in sentence2:
        if word in stopwords:
            continue # do nothing
        
        # add 1 to vector's index corresponding to word
        vector2[all_words.index(word)] += 1
            
    return 1 - cosine_distance(vector1, vector2)

In [11]:
def sentence_similarity_matrix(sentences, stopwords=None):
    """
    sentences = [['word1, 'word2', ...], [...], ...]
    """
    
    # create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                continue # do nothing
                
            similarity_matrix[i][j] = sentence_similarity(sentences[i], sentences[j], stopwords)
            
    # row-wise matrix normalization to penalize longer sentences
    for i in range(len(similarity_matrix)):
        similarity_matrix[i] /= similarity_matrix[i].sum()

    return similarity_matrix

In [12]:
def page_rank(similarity_matrix, eps=0.0001, d=0.85):
    """
    similarity_matrix[i][j] = probability of transitioning from sentence i to sentence j
    eps = stop the algorithm when the difference between two consecutive iterations <= eps
    d (damping factor) = with a probability 1-d the user will simply pick a sentence at random as the next destination, ignoring the structure completely    
    """
    
    print('eps: {}'.format(eps))
    
    p = np.ones(len(similarity_matrix)) / len(similarity_matrix)
    
    while True:
        new_p = np.ones(len(similarity_matrix)) * ((1 - d) / len(similarity_matrix)) + d * similarity_matrix.T.dot(p)
        delta = abs(new_p - p).sum()
        print('delta: {}'.format(delta))
        
        if delta <= eps:
            return new_p
        else:
            p = new_p

In [13]:
def rank_text(sentences, top=5, stopwords=None):
    """
    sentences = a list of sentences [[w11, w12, ...], [w21, w22, ...], ...]
    top = how may sentences the summary should contain
    stopwords = a list of stopwords
    """
    
    similarity_matrix = sentence_similarity_matrix(sentences, stopwords)
    
    sentence_ranks = page_rank(similarity_matrix)
    
    # sort sentence ranks
    ranked_sentence_indices = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indices[:top])
    
    return [' '.join(sentence) for i, sentence in enumerate(itemgetter(*selected_sentences)(sentences))]

In [14]:
i = 1
print(data[i])
print(rank_text(preprocessed_data[i], top=3, stopwords=stopwords.words('english')))

Frédéric François Chopin (/ˈʃoʊpæn/; French pronunciation: ​[fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃]; 22 February or 1 March 1810 – 17 October 1849), born Fryderyk Franciszek Chopin,[n 1] was a Polish and French (by citizenship and birth of father) composer and a virtuoso pianist of the Romantic era, who wrote primarily for the solo piano. He gained and has maintained renown worldwide as one of the leading musicians of his era, whose "poetic genius was based on a professional technique that was without equal in his generation." Chopin was born in what was then the Duchy of Warsaw, and grew up in Warsaw, which after 1815 became part of Congress Poland. A child prodigy, he completed his musical education and composed his earlier works in Warsaw before leaving Poland at the age of 20, less than a month before the outbreak of the November 1830 Uprising.
eps: 0.0001
delta: 0.5666666666666667
delta: 0.4816666666666667
delta: 0.40941666666666676
delta: 0.3480041666666668
delta: 0.29580354166666684
delta:

### Multi-word Phrase Extraction

In [16]:
from nltk.tag.stanford import StanfordPOSTagger
from stanford_postagger.stanford_wrapper import StanfordPOSTagger as StanfordPOSTaggerWrapper

#### Add POS tags to words

In [17]:
postagger = StanfordPOSTaggerWrapper()

In [31]:
def add_postags(dataset):
    postagged_paragraphs = []
    for paragraph in dataset:
        postagged_paragraph = []
        for sentence in paragraph:
            postagged_sentence = []
            for word in sentence:
                postagged_word = postagger.tag(word)
                postagged_sentence.append(postagged_word[0])
            postagged_paragraph.append(postagged_sentence)
        postagged_paragraphs.append(postagged_paragraph)
    
    return postagged_paragraphs

In [32]:
postagged_data = add_postags(preprocessed_data)

In [42]:
postagged_data

[[[('Beyoncé', 'NNP'),
   ('Giselle', 'NNP'),
   ('Knowles-Carter', 'NNP'),
   ('bee-YON-say', 'NN'),
   ('born', 'VBN'),
   ('September', 'NNP'),
   ('1981', 'CD'),
   ('is', 'VBZ'),
   ('an', 'DT'),
   ('American', 'NNP'),
   ('singer', 'NN'),
   ('songwriter', 'NN'),
   ('record', 'NN'),
   ('producer', 'NN'),
   ('and', 'CC'),
   ('actress', 'NN')],
  [('Born', 'VBN'),
   ('and', 'CC'),
   ('raised', 'VBN'),
   ('in', 'IN'),
   ('Houston', 'NNP'),
   ('Texas', 'NNP'),
   ('she', 'PRP'),
   ('performed', 'VBN'),
   ('in', 'IN'),
   ('various', 'JJ'),
   ('singing', 'NN'),
   ('and', 'CC'),
   ('dancing', 'NN'),
   ('competitions', 'NNS'),
   ('as', 'IN'),
   ('a', 'DT'),
   ('child', 'NN'),
   ('and', 'CC'),
   ('rose', 'VBD'),
   ('to', 'TO'),
   ('fame', 'NN'),
   ('in', 'IN'),
   ('the', 'DT'),
   ('late', 'RB'),
   ('1990s', 'NNS'),
   ('as', 'IN'),
   ('lead', 'NN'),
   ('singer', 'NN'),
   ('of', 'IN'),
   ('R&B', 'NN'),
   ('girl-group', 'NN'),
   ('Destiny', 'NN'),
   ('Chil

In [43]:
def find_all(element, array):
    indices = [i for i, x in enumerate(array) if x == element]
    return indices

In [ ]:
def do_they_occur_together(word1, word2, window_size, sentence):
    indices1 = find_all(word1, sentence)
    indices2 = find_all(word2, sentence)
    
    found = false
    while not found and :
    